In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

# prevent tensorflow from using GPU. Otherwise, run out of memory
# https://stackoverflow.com/questions/44552585/prevent-tensorflow-from-accessing-the-gpu
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import tensorflow_hub as hub
import tensorflow as tf

import nltk
import regex as re
import emoji as em

import itertools as it

elmo = hub.Module("https://tfhub.dev/google/elmo/2")

W0416 14:45:01.628971 140044708812608 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
# Load Pickles
pickle_path = '/home/tim/Documents/Sentiment/Data/pickles'

#x_train
pickle_in = open(os.path.join(pickle_path,'elmo_embeddings.pickle'),"rb")
elmo_dict = pickle.load(pickle_in)

In [6]:
print("elmo_dict length:", len(elmo_dict))

elmo_dict length: 1193514


In [7]:
with open('elmo.txt', 'w') as f:
    i = 0
    for key, value in elmo_dict.items():
        
        
        f.write('%s\t' % (key))
        for x in value:
            f.write('%s ' % (x))
        i += 1
        if i %10000 == 0:
            print("key:\t", i)
        f.write('\n')

key:	 10000
key:	 20000
key:	 30000
key:	 40000
key:	 50000
key:	 60000
key:	 70000
key:	 80000
key:	 90000
key:	 100000
key:	 110000
key:	 120000
key:	 130000
key:	 140000
key:	 150000
key:	 160000
key:	 170000
key:	 180000
key:	 190000
key:	 200000
key:	 210000
key:	 220000
key:	 230000
key:	 240000
key:	 250000
key:	 260000
key:	 270000
key:	 280000
key:	 290000
key:	 300000
key:	 310000
key:	 320000
key:	 330000
key:	 340000
key:	 350000
key:	 360000
key:	 370000
key:	 380000
key:	 390000
key:	 400000
key:	 410000
key:	 420000
key:	 430000
key:	 440000
key:	 450000
key:	 460000
key:	 470000
key:	 480000
key:	 490000
key:	 500000
key:	 510000
key:	 520000
key:	 530000
key:	 540000
key:	 550000
key:	 560000
key:	 570000
key:	 580000
key:	 590000
key:	 600000
key:	 610000
key:	 620000
key:	 630000
key:	 640000
key:	 650000
key:	 660000
key:	 670000
key:	 680000
key:	 690000
key:	 700000
key:	 710000
key:	 720000
key:	 730000
key:	 740000
key:	 750000
key:	 760000
key:	 770000
key:	 78

In [ ]:
t = elmo_dict['<user>']

In [ ]:
for x in t:
    print(x)

In [ ]:
for key, value in elmo_dict.items():
    print(value)

In [ ]:
import json


with open('file.txt', 'w') as file:
     file.write(json.dumps(elmo_dict))

In [ ]:
# FUNCTIONS

# process tweet
def preprocess(tweet):
    #define hashtag pattern
    hashtag_define = re.compile ('#')
    #define mention pattern 
    mentions_define = re.compile('@[^\s]+')
    #define link pattern
    link_define = re.compile('https?://[^\s]+')
    # Haystack define
    haystack_define = re.compile('(RT)')
    # define long spaces
    extra_spaces_define = re.compile('\s{2,}')
    #remove hashtags
    tweet_refine = hashtag_define.sub('',tweet)
    # remove mentions
    tweet_refine = mentions_define.sub('',tweet_refine)
    # remove links
    tweet_refine = link_define.sub('',tweet_refine)
    # remove haystack
    tweet_refine = haystack_define.sub('',tweet_refine)
    #replace long spaces with one space
    tweet_refine = extra_spaces_define.sub(' ',tweet_refine)
    # convert emoticons into words
    tweet_refine = em.demojize(tweet_refine)
    return tweet_refine

# Create tokens with simple split on whitespace
def simple_token(s):
    return s.split()

# Get token length
def token_length(s):
    return len(s)

# Create "embedding lists" of equal size -- pad with empty characters, e.g. ""
# https://stackoverflow.com/questions/24066904/most-pythonic-way-to-extend-a-list-to-exactly-a-certain-length
def pad_list(some_list, target_len):
    return some_list[:target_len] + [""]*(target_len - len(some_list))

# Get the elmo embeddings
def elmo_tweet_embedder(tokens,len_list):
    
    tokens_input = tokens #load a tweet
    tokens_length = len_list # get length of tweet

    #create embedding
    embedding_tensor = elmo(inputs={"tokens":tokens_input,"sequence_len":tokens_length},
                            signature="tokens", as_dict=True)["word_emb"] # <-- passing in a list instead of [word]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        embedding = sess.run(embedding_tensor)
        return embedding

# combine all together
def make_embeddings(df, max_seq_len):
    m = max_seq_len
    
    # create a token column in df
    df['tokens'] = df['text'].apply(simple_token)
    print("Token creation complete")
    
    # pad the tokens
    df['tokens'] = df.apply(lambda x: pad_list(x['tokens'],m),axis=1)
    print("Token padding complete")
    
    # split data into smaller batches of size 100. http://bit.ly/2P4J8HJ
    text_batches = [df['tokens'][i:i+5000] for i in range(0,df.shape[0],5000)]
    len_lists = [[m] * len(x) for x in text_batches]

    # create list of sentiments (y values)
    y = df['sent'].to_numpy()
    
    # make embeddings
    embeddings = []
    for i in range(0,len(text_batches)):
        print("batch",i)
        elmo_train = elmo_tweet_embedder(text_batches[i].tolist(),len_lists[i])
        embeddings.append(elmo_train)
        
    embeddings = np.concatenate(embeddings, axis=0)
    
    return embeddings, y

In [ ]:
#ORIGINAL

path = '/home/tim/Documents/Sentiment/Data/dictionary_raw'
file = 'vocab_complete.txt'

# path = '/home/tim/Documents/Sentiment/Data/dictionary_raw'
# file = 'text.txt'

p = os.path.join(path,file)

# open file
f = open(p, "r")
lines = f.readlines()

#remove new line symbol
l=[]
for x in lines:
    x = x.rstrip("\n")
    l.append(x)
    
# create df
col_names = ['sent']
df = pd.DataFrame(l,columns=col_names)
df['text']=df['sent']
f.close()
df.head()

In [ ]:
len(df['sent'])

In [ ]:
X, y = make_embeddings(df, 1)
d = dict()
for i in range(len(y)):
    d[y[i]] = X[i][0]

In [ ]:
# Create Pickles
pickle_path = '/home/tim/Documents/Sentiment/Data/pickles'

# elmo embeddings
pickle_out = open(os.path.join(pickle_path,'elmo_embeddings.pickle'),"wb")
pickle.dump(d,pickle_out)
pickle_out.close()